In [8]:
import pandas as pd
import numpy as np
import ipywidgets
import matplotlib.pyplot as plt
import plotly.express as px
from IPython.display import display

In [9]:
df_le_sex = pd.read_csv('full-data/life_expectancy_by_sex.csv')

In [10]:
all_countries = df_le_sex.loc[df_le_sex['Entity'] != 'World', ['Entity']]['Entity'].unique()
all_countries = [str(i) for i in all_countries]

In [11]:
def show_line_chart_gender(country):
    fig = plt.figure(figsize=(12, 5))
    ax1 = plt.subplot(1,1,1)
    x_labels_official = [1950, 1960, 1970, 1980, 1990, 2000, 2010, 2021]
    y_labels = [30, 40, 50, 60, 70, 80]
    colors = {'men':'blue', 'women':'red'}
    tmp = df_le_sex.loc[df_le_sex['Entity'] == country, ['Life_exp_female', 'Life_exp_male', 'Year']]
    x_male = tmp.Life_exp_male.values
    x_female = tmp.Life_exp_female.values
    y = tmp.Year.values
    ax1.plot(y, x_male, '.-', color=colors['men'], label='nam')
    ax1.plot(y, x_female, '.-', color=colors['women'], label='nữ')
    # set up plot
    ax1.set_title("Life expectancy so sánh giữa nam và nữ tại " + country)
    ax1.set_xticks(x_labels_official)
    ax1.set_yticks(y_labels)
    ax1.spines['top'].set_visible(False)
    ax1.spines['right'].set_visible(False)
    y_ticks = ax1.get_yticks()
    for y_tick in y_ticks:
        ax1.axhline(y=y_tick, color='#E7FCFB', linestyle='--', zorder=-1)
    plt.xlabel("Năm")
    plt.ylabel("Life expectancy")
    plt.legend(loc = "upper left")
    plt.show()

In [12]:
# Create the animation widget
animation4 = ipywidgets.interact(show_line_chart_gender, 
                                 country=ipywidgets.Combobox(value='World', options=all_countries, description='Country:', ensure_option=True,disabled=False))
# Display the animation widget
display(animation4)

interactive(children=(Combobox(value='World', description='Country:', ensure_option=True, options=('Afghanista…

<function __main__.show_line_chart_gender(country)>

In [13]:
def normalize(numbers, min_val, max_val, range):
    range_val = max_val - min_val
    normalized = []
    for num in numbers:
        normalized_val = (num - min_val) / range_val
        normalized_val = normalized_val * (range[1] - range[0]) + range[0] # normalize to range
        normalized.append(normalized_val)
    return normalized

In [14]:
all_year_2 = sorted(df_le_sex.Year.unique())
all_year_2 = [str(i) for i in all_year_2]
all_text_2 = ['All', 'Africa', 'Americas', 'Asia', 'Europe', 'Oceania']
all_countries = df_le_sex.loc[df_le_sex['Entity'] != 'World', ['Entity']]['Entity'].unique()
all_countries = [str(i) for i in all_countries]

In [18]:
def show_scatter_plot_gender(year, text):
    print('Each dot is a country. And the size of the dot refer to the population in a country')
    colors = {'Africa':'#D1AFE4', 'Americas':'#2587F0', 'Asia':'#21DA61', 'Europe':'#E9C213', 'Oceania':'#F791C1'}
    groups = ['Africa', 'Americas', 'Asia', 'Europe', 'Oceania']
    translate = {'World': 'Thế giới', 'Oceania': 'Châu Úc', 'Europe': 'Châu Âu', 'Americas': 'Châu Mỹ', 'Asia': 'Châu Á', 'Africa': 'Châu Phi'}
    year = int(year)
    small_grouped_df = df_le_sex.groupby('Entity')
    fig = plt.figure(figsize=(10, 10))
    ax1 = plt.subplot(1,1,1)
    x_arr, y_arr, size, continents = [], [], [], []
    if text == 'All':
        valid_con = ['Africa', 'Americas', 'Asia', 'Europe', 'Oceania']
    else:
        valid_con = [text]
    for key, df_local in small_grouped_df:
        tmp = df_local.loc[df_local['Year'] == year, ['Life_exp_male', 'Life_exp_female', 'Population', 'Entity', 'Continent']]
        if len(tmp) == 0:
            continue
        x = tmp.Life_exp_male.values[0]
        y = tmp.Life_exp_female.values[0]
        p = tmp.Population.values[0]
        con = tmp.Continent.values[0]
        #if con not in groups:
            #continue
        x_arr.append(x)
        y_arr.append(y)
        size.append(p)
        continents.append(con)
    # normalize size
    norm_size = normalize(size, min(size), max(size), [100, 2000])
    # set up scatter plot
    for g in groups:
        indexes = [i for i in range(len(continents)) if continents[i] == g]
        tmp_x = []
        tmp_y = []
        tmp_s = []
        for i in indexes:
            tmp_x.append(x_arr[i])
            tmp_y.append(y_arr[i])
            tmp_s.append(norm_size[i])
        if g in valid_con:
            scatter = ax1.scatter(x=tmp_x, y=tmp_y, s=tmp_s, c=colors[g], alpha=0.7, edgecolors='none', label=translate[g])
        else:
            scatter = ax1.scatter(x=tmp_x, y=tmp_y, s=tmp_s, c='#EEEEEE', alpha=0.7, edgecolors='none', zorder=-1)
    lims = [
        np.min([ax1.get_xlim(), ax1.get_ylim()]),  # min of both axes
        np.max([ax1.get_xlim(), ax1.get_ylim()]),  # max of both axes
    ]

    # now plot both limits against eachother
    ax1.plot(lims, lims, linestyle='--', color='gray', alpha=0.6, zorder=-1)
    ax1.set_title("Tuổi thọ dự kiến của phụ nữ và đàn ông trong năm " + str(year))
    lgnd = ax1.legend(loc="lower right", numpoints=1, title='Châu lục', fontsize=10)
    
    #change the marker size manually for both lines
    if text == 'All':
        lgnd.legendHandles[0]._sizes = [90]
        lgnd.legendHandles[1]._sizes = [90]
        lgnd.legendHandles[2]._sizes = [90]
        lgnd.legendHandles[3]._sizes = [90]
        lgnd.legendHandles[4]._sizes = [90]
    else:
        lgnd.legendHandles[0]._sizes = [90]
    h, l = plt.gca().get_legend_handles_labels()
    plt.xlabel("Tuổi thọ dự kiến của đàn ông (đv: năm)")
    plt.ylabel("Tuổi thọ dự kiến của phụ nữ (đv: năm)")
    plt.show()

In [19]:
# Create the animation widget
animation3 = ipywidgets.interact(show_scatter_plot_gender, 
                                 year=ipywidgets.Combobox(value='2021', options=all_year_2, description='Year:', ensure_option=True,disabled=False),
                                 text=ipywidgets.Combobox(value='All', options=all_text_2, description='Mode:', ensure_option=True,disabled=False))
# Display the animation widget
display(animation3)

interactive(children=(Combobox(value='2021', description='Year:', ensure_option=True, options=('1950', '1951',…

<function __main__.show_scatter_plot_gender(year, text)>

In [17]:
# Create the animation widget
animation5 = ipywidgets.interact(show_line_chart_gender, 
                                 country=ipywidgets.Combobox(value='Vietnam', options=all_countries, description='Country:', ensure_option=True,disabled=False))
# Display the animation widget
display(animation5)

interactive(children=(Combobox(value='Vietnam', description='Country:', ensure_option=True, options=('Afghanis…

<function __main__.show_line_chart_gender(country)>